In [2]:
import gzip
from collections import defaultdict
import sklearn
from sklearn import linear_model
import string
import re
import datetime, time
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as md
import math
from gensim.models import Word2Vec


In [3]:
def readCSV(path):
    f = open(path, 'rt')
    f.readline()

    for l in f:
        
        if l.startswith("timestamp"): 
            continue

        try: 
            timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education = re.split(r',(?![ ])', l)

        except:
            continue

        yield timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education


In [4]:
salaryRaw = []
for l in readCSV("Levels_Fyi_Salary_Data.csv"):
    salaryRaw.append(l)

In [5]:
print(len(salaryRaw))
#city id = 14, dmaid = 15, city = 5
print(salaryRaw[0][5])

print(salaryRaw[0])



62300
"Redwood City, CA"
('6/7/2017 11:33:27', 'Oracle', 'L3', 'Product Manager', '127000', '"Redwood City, CA"', '1.5', '1.5', 'NA', '107000', '20000', '10000', 'NA', 'NA', '7392', '807', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'NA', 'NA\n')


In [6]:
# id , unixtime, company, level, title, total_comp, city, state, experience, tenure, tag, 
# salary, stock, bonus, city_id, dma_id, ms_deg, bs_deg, phd_deg, hs, college

def parseSalaryRaw(salaryRaw):
    salaryAllData = []

    for entry in salaryRaw:

        date, clocktime = entry[0].split(' ')
        date = date.split('/')
        clocktime = clocktime.split(':')
        timestamp = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(clocktime[0]), int(clocktime[1]), int(clocktime[2]))
        timestamp = int(time.mktime(timestamp.timetuple()))


        company = entry[1]
        level = entry[2]
        title = entry[3]

        total_comp = int(float(entry[4]))

        location = entry[5]
        if location.count(',') == 2:
            city, state, country = location.strip('"').split(', ')
        elif location.count(',') == 1:
            city, state = location.strip('"').split(', ')
            country = 'USA'

        experience = entry[6]
        tenure = entry[7]

        tag = entry[8]

        salary = int(float(entry[9]))
        if salary == 0:
            continue

        stock = int(float(entry[10]))
        bonus = int(float(entry[11]))

        g = entry[12]
        gender = -1
        if g == 'Female':
            gender = 0
        elif g == 'Male':
            gender = 1
        elif g == 'Other':
            gender = 2

        city_id = int(entry[14])

        try: 
            dma_id = int(entry[15])
        except:
            dma_id = -1

        id = int(entry[16])

        ms_deg = int(entry[17])
        bs_deg = int(entry[18])
        phd_deg = int(entry[18])
        hs = int(entry[19])
        college = int(entry[20])

        r = entry[27]
        race = -1
        if r == 'Asian':
            race = 0
        elif r == 'Black':
            race = 1
        elif r == 'Hispanic':
            race = 2
        elif r == 'Two Or More':
            race = 3
        elif r == 'White':
            race = 4


        salaryAllData.append({
            'id': id,
            'timestamp': timestamp,
            'company': company,
            'level': level,
            'title': title,
            'total_comp': total_comp,
            'city': city,
            'state': state,
            'country': country,
            'experience': experience,
            'tenure': tenure,
            'tag': tag,
            'salary': salary,
            'stock': stock,
            'bonus': bonus,
            'gender': gender,
            'city_id': city_id,
            'dma_id': dma_id,
            'ms_deg': ms_deg,
            'bs_deg': bs_deg,
            'phd_deg': phd_deg,
            'hs': hs,
            'college': college,
            'race': race,
        })
    return salaryAllData

salaryAllData = parseSalaryRaw(salaryRaw)

In [7]:
print(len(salaryAllData))
print(salaryAllData[4]['dma_id'])

59996
819


In [8]:
#####
# Training, Validation, and Test sets
#####

# shuffle data
shuffledAllData = sklearn.utils.shuffle(salaryAllData)
shuffledAllData[0]['id']

# create 80 / 10 / 10 : Train / Validation / Test sets
X = shuffledAllData
y = [d['salary'] for d in shuffledAllData]

X_train, X_temp, y_train, y_temp = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = sklearn.model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Validate that sets were created with propper sizes
print(str(len(X_train)) + ", " + str(len(y_train)))
print(str(len(X_valid)) + ", " + str(len(y_valid)))
print(str(len(X_test)) + ", " + str(len(y_test)))

# Validate all ids are different
print(str(X_train[0]['id']) + ", " + str(X_valid[0]['id']) + ", " + str(X_test[0]['id']))

47996, 47996
6000, 6000
6000, 6000
80544, 6524, 21074


In [9]:
X_train[0]

{'id': 80544,
 'timestamp': 1627594982,
 'company': 'Apple',
 'level': 'ICT3',
 'title': 'Software Engineer',
 'total_comp': 220000,
 'city': 'Cupertino',
 'state': 'CA',
 'country': 'USA',
 'experience': '2',
 'tenure': '2',
 'tag': 'ML / AI',
 'salary': 140000,
 'stock': 80000,
 'bonus': 0,
 'gender': 0,
 'city_id': 7158,
 'dma_id': 807,
 'ms_deg': 0,
 'bs_deg': 1,
 'phd_deg': 1,
 'hs': 0,
 'college': 0,
 'race': 0}

In [10]:
#counting all dma_id just for exploration
#assigning indices for every dma_id
def dma_count_plus_index(dict, ind_dict):
    count = 1
    for d in salaryRaw:
        dict[d[15]] += 1
        if not ind_dict.get(d[15]):
            ind_dict[d[15]] = count
            count += 1
        


In [11]:
dmaidcounts = defaultdict(int)
dmaindices = {}
dma_count_plus_index(dmaidcounts, dmaindices)

In [12]:
print(dmaindices)

{'807': 1, '819': 2, '0': 3, '501': 4, '512': 5, '862': 6, '820': 7, '803': 8, '511': 9, '635': 10, '506': 11, '825': 12, '622': 13, '508': 14, '519': 15, '751': 16, '652': 17, '504': 18, '527': 19, '617': 20, '613': 21, '602': 22, '532': 23, '753': 24, '752': 25, '623': 26, '556': 27, '669': 28, '628': 29, '566': 30, '515': 31, '560': 32, '528': 33, '534': 34, '524': 35, '609': 36, '526': 37, '521': 38, '648': 39, '770': 40, '517': 41, '518': 42, '535': 43, '505': 44, '641': 45, '790': 46, '539': 47, '670': 48, '533': 49, '523': 50, '618': 51, '637': 52, '616': 53, '855': 54, '510': 55, '563': 56, '757': 57, '659': 58, '625': 59, '691': 60, '705': 61, '576': 62, '811': 63, '679': 64, '541': 65, '544': 66, '828': 67, '789': 68, '821': 69, '588': 70, '611': 71, '716': 72, '801': 73, '561': 74, '682': 75, '584': 76, '671': 77, 'NA': 78, '564': 79, '502': 80, '640': 81, '678': 82, '581': 83, '642': 84, '514': 85, '598': 86, '643': 87, '567': 88, '839': 89, '746': 90, '500': 91, '675': 92,

In [13]:
def dma_feat(d, dict):  #returns feature row for dma_id one hot ecoding only
    arr = [1] + ([0]*149)
    if d == '-1':
         arr[77] = 1
    elif d != '807':
        arr[(dict[d]-1)] = 1
    return arr

def dma_yoe_feat(d, dict, exp): #returns row for dma_id and either tenure/yoe
    # print(exp, type(exp))
    arr = [1] + ([0]*149)
    if d == '-1':
         arr[77] = 1
    elif d != '807':
        arr[(dict[d]-1)] = 1
    return arr + [exp]

In [14]:
#constructing feature vectors

X_train_dma = [dma_feat(str(d['dma_id']), dmaindices) for d in X_train]
X_train_dma_yoe = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['experience']))) for d in X_train]
X_train_dma_tenure = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['tenure']))) for d in X_train]


In [15]:
#first model with just dma_id one-hot encoding
model = sklearn.linear_model.LinearRegression(fit_intercept=False)
model.fit(X_train_dma, y_train)
predictions = model.predict(X_train_dma)

In [16]:
#second model with dma_id one-hot encoding + yoe (exeperience)
model2 = sklearn.linear_model.LinearRegression(fit_intercept=False)
model2.fit(X_train_dma_yoe, y_train)
predictions2 = model2.predict(X_train_dma_yoe)

In [17]:
#third model with dma_id one-hot encoding + tenure
model3 = sklearn.linear_model.LinearRegression(fit_intercept=False)
model3.fit(X_train_dma_tenure, y_train)
predictions3 = model3.predict(X_train_dma_tenure)

In [18]:
print(predictions[:10])
print(predictions2[:10])
print(predictions3[:10])
y_train[:10]

[172110.76604408 122143.50797267 151289.85975414  80311.85311475
 137293.62170088 172110.76604408  80311.85311475 172110.76604408
 130001.09114383 172110.76604408]
[150339.37411287 105832.12576975 132873.78105907 113310.89205963
 142284.0766659  154452.35331169  92745.99606554 142113.41571523
 157419.79597631 154452.35331169]
[170445.61884799 116914.19117501 151486.07230669  85462.69426457
 128922.62688315 168811.10829322  75655.63093596 163907.57662892
 127820.65259361 173714.63995753]


[140000, 109000, 187000, 100000, 160000, 175000, 77000, 121000, 160000, 130000]

In [19]:
# for key,value in dmaindices.items():
    
#     print(key, value)

In [20]:
# print(len(dmaidcounts))

# dmaidcounts['807']

In [21]:
# for key,value in dmaidcounts.items():
#     print(key, value)

In [22]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [23]:
def percent_err(y, pred):
    tot = 0
    for actual, pred in zip(y, pred):
        tot += abs(100 * ((pred-actual) / pred))
    avg = tot/len(y)
    return avg

In [24]:
#create feature vector for X_valid

X_valid_dma = [dma_feat(str(d['dma_id']), dmaindices) for d in X_valid]
X_valid_dma_yoe = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['experience']))) for d in X_valid]
X_valid_dma_tenure = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['tenure']))) for d in X_valid]


In [25]:
valid_predictions_dma = model.predict(X_valid_dma)
valid_predictions_dma_yoe = model2.predict(X_valid_dma_yoe)
valid_predictions_dma_tenure = model3.predict(X_valid_dma_tenure)


In [26]:
print(valid_predictions_dma[:10])

print(valid_predictions_dma_yoe[:10])

print(valid_predictions_dma_tenure[:10])

print(y_valid[:10])

[172110.76604408  80311.85311475 133251.78907721  80311.85311475
 151289.85975414 114580.41958042 172110.76604408 172110.76604408
 172110.76604408 172110.76604408]
[154452.35331169  80407.05846909 113833.26016513  80407.05846909
 153438.67705316 105577.88468328 146226.39491405 150339.37411287
 154452.35331169 162678.31170932]
[163907.57662892  75655.63093596 132762.35936307  75655.63093596
 154755.09341623 103476.03493736 167176.59773846 167176.59773846
 167176.59773846 163907.57662892]
[135000, 132000, 85000, 35000, 180000, 124000, 128000, 200000, 140000, 200000]


In [27]:
percent_err(y_valid, valid_predictions_dma )

24.495947880469995

In [28]:
MSE(y_valid, valid_predictions_dma )

2157142942.845791

In [29]:
print(percent_err(y_valid, valid_predictions_dma_yoe))
MSE(y_valid, valid_predictions_dma_yoe)




20.160186682126422


1590097706.8308477

In [30]:
print(percent_err(y_valid, valid_predictions_dma_tenure))
MSE(y_valid, valid_predictions_dma_tenure)

23.865535845509847


2058551558.4781947